<a href="https://colab.research.google.com/github/tanzeem131/AI-Agent-for-websiteQnA/blob/main/AI-Agent-for-websiteQnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community
!pip install langchain-google-genai google-generativeai
!pip install -U duckduckgo-search

In [2]:
import os
from google.colab import userdata

import requests
from bs4 import BeautifulSoup

# For creating AI agents
from langchain.agents import initialize_agent,Tool,AgentType
# from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
os.environ["GOOGLE_API_KEY"] = userdata.get("gemini")

Extract text from URL

In [4]:
def extract_website_text(url):
  response=requests.get(url)
  soup=BeautifulSoup(response.text,"html.parser")
  return soup.get_text()

Creating Q and A for tool for agent

In [5]:
#define func for processing user input
def website_qa_tool(url_and_question):
  url,question=url_and_question.split('|',1)
  content=extract_website_text(url.strip())
  prompt=f"Anser this question using the content in the website:\n\n{question.strip()}\n\nContent:\n{content[:6000]}"
  # llm=ChatOpenAI()
  llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
  return llm.predict(prompt)

#creating tool for agent
tool = Tool(
    name="WebsiteQnA",
    func=website_qa_tool,
    description="Input format: '<Website URL>|<your questions>'. Use this tool to ask questions about website's content."
)


Initializing the Agent

In [6]:
agent=initialize_agent(
    tools=[tool],
    # llm=ChatOpenAI(),
    llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash"),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

/tmp/ipython-input-340985304.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(


Running the agent

In [7]:
print(agent.run("https://www.ibm.com/think/topics/machine-learning | Explain Machine learning in short."))

/tmp/ipython-input-1564996917.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(agent.run("https://www.ibm.com/think/topics/machine-learning | Explain Machine learning in short."))




> Entering new AgentExecutor chain...
I need to summarize the content of the given URL to explain machine learning in short.
Action: WebsiteQnA
Action Input: https://www.ibm.com/think/topics/machine-learning | Explain Machine learning in short.

/tmp/ipython-input-2346778668.py:8: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm.predict(prompt)



Observation: Machine learning (ML) is a branch of artificial intelligence (AI) that focuses on enabling computers and machines to imitate how humans learn. This allows them to perform tasks autonomously and improve their performance and accuracy through experience and exposure to more data. ML algorithms make predictions or classifications based on input data, evaluate these predictions using an error function, and then optimize the model by adjusting weights until a desired level of accuracy is achieved.
Thought:I now know the final answer
Final Answer: Machine learning is a branch of AI that enables computers to learn from data, improve their performance, and make predictions or classifications without explicit programming.

> Finished chain.
Machine learning is a branch of AI that enables computers to learn from data, improve their performance, and make predictions or classifications without explicit programming.
